In [ ]:
# pymatgen == 2022.8.23

In [ ]:
from pymatgen.core import Structure
from pymatgen.analysis.local_env import CrystalNN, VoronoiNN
from pymatgen.analysis.structure_matcher import StructureMatcher, FrameworkComparator
import random, json, os, pickle
import numpy as np
from tqdm import tqdm

In [ ]:
def get_trimmed_structure(s) : 
    """
    Returns pymatgen.core.Structure without frac_coords == 1.
    If fractional coordinate of an atom = 1, then it will be trimmed to 0.
    """
    for i, atom in enumerate(s) : 
        if 1 in np.round(atom.frac_coords, 4) : 
            while 1 in np.round(atom.frac_coords, 4) : 
                index_1 = np.round(atom.frac_coords, 4).tolist().index(1)
                s[i].frac_coords[index_1] = 0
            
        else : 
            pass
        
    return s

In [ ]:
def is_edge_sharing(s, index_list) :
    """
    Returns True if two sites are edge-sharing position. (In Rocksalt structure)
    s : pymatgen.core.Structure
    index_list : 1X2 type list contains indexes of two sites.
    """
    cnn = CrystalNN()
    info_1, info_2 = cnn.get_nn_info(s, index_list[0]), cnn.get_nn_info(s, index_list[1])
    common = [x for x in info_1 if x in info_2 and x['site'].specie.name == "O"] # in case of Oxide Rocksalt
    
    if len(common) == 2 : 
        return True
    else : 
        return False

def is_corner_sharing(s, index_list) :
    """
    Returns True if two sites are corner-sharing position. (In Rocksalt structure)
    s : pymatgen.core.Structure
    index_list : 1X2 type list contains indexes of two sites.
    """
    cnn = CrystalNN()
    info_1, info_2 = cnn.get_nn_info(s, index_list[0]), cnn.get_nn_info(s, index_list[1])
    common = [x for x in info_1 if x in info_2 and x['site'].specie.name == "O"] # in case of Oxide Rocksalt
    
    if len(common) == 1 : 
        return True
    else : 
        return False 

In [ ]:
def get_percolation_on_radius(s, index, percolating_species, radius) : 
    """
    Returns site indices which is neighboring with site of 'index' (s[i])
    Do not contains given 'index' itself and Periodic Boundary Condition is not considered.
    Note that this function determines percolation ONLY BASED ON RADIUS.
    (atoms in shell of radius = percolated cluster)
    s : pymatgen.core.Structure
    index : Integer
    radius = Cutoff radius, which is criterion that determines neighboring atoms
    """
    neighbor = [n for n in s.get_neighbors(s[index], r = radius) if n.specie.name == percolating_species and (n.frac_coords >= 0).all() == True]
    neighbor_index = [i for n in neighbor for i in range(len(s)) if s[i] == n]
            
    return neighbor_index

In [ ]:
#  DO NOT CONSIDER Periodic Boundary Condition ==> a=1 (direct coordination) in unit cell => do not considered
def get_percolating_cluster_radius_from_structure(s, percolating_species, radius) :
    """
    Returns percolating clusters based on PERCOLATION RADIUS ONLY.
    s : pymatgen.core.Structure
    percolating_species : string data which is percolating atomic symbol
    radius : percolation radius. species except oxygen located inside of shell of 'radius' are 
             treated as percolated
    
    Ex) s : Li10Mn6O16, percolating_species : 'Mn'
        ==> returns percolating cluster in Li10Mn6O16 DRX structure
    """
    s = get_trimmed_structure(s) 
    percolating_cluster = []
    percolating_species_index = [i for i in range(len(s)) if s[i].specie.name == percolating_species]
    remains = percolating_species_index.copy()

    while len(remains) != 0 : 
        cluster = []

        new_index = remains[0]
        cluster.append(new_index)

        neighbor = [x for x in get_percolation_on_radius(s, new_index, percolating_species, radius) if x not in cluster]
        
        cluster = cluster + neighbor
        remains = [x for x in remains if x not in cluster] 

        while neighbor :
            new_neighbor = []
            for i in neighbor : 
                new_neighbor = new_neighbor + [x for x in get_percolation_on_radius(s, i, percolating_species, radius) if x not in cluster and x not in new_neighbor]

            cluster = cluster + new_neighbor
            remains = [x for x in remains if x not in new_neighbor]
            neighbor = new_neighbor

        percolating_cluster.append(cluster)
    
    return percolating_cluster

In [ ]:
def is_cluster_percolated(s, cluster, axis) : 
    """
    Returns True if cluster actually contribute to percolation along certain direction(axis)
    s : pymatgen.core.Structure
    cluster : list of indexes of percolated atoms (Ex : [1,2,6,8,10,13,16])
    axis : percolation direction ('a'(a-axis), 'b'(b-axis), or 'c'(c-axis))
    """
    if axis == 'a' : 
        ax = 0
    elif axis == 'b' : 
        ax = 1
    elif axis == 'c' : 
        ax = 2
        
    coords = [round(site.frac_coords[ax], 2) for site in s if site.specie.name != 'O' and site.specie.name != 'F']
    max_coord, min_coord = max(coords), min(coords)
    bottom, top = [i for i in range(len(s)) if round(s[i].frac_coords[ax], 2) == min_coord], [i for i in range(len(s)) if round(s[i].frac_coords[ax], 2) == max_coord]
    
    percolated = bool(list(set(cluster) & set(bottom))) and bool(list(set(cluster) & set(top)))
    
    return percolated

def is_dead_cluster(s, cluster) :
    """
    Returns True if given cluster do not contribute to entire percolation, that means, the cluster do not
    percolated along overall a, b, and c-axis.
    s : pymatgen.core.Structure
    cluster : list of indexes of percolated atoms (Ex : [1,2,6,8,10,13,16])
    """
    percolated = [is_cluster_percolated(s, cluster, axis) for axis in ['a', 'b', 'c']]
    is_dead = not(percolated[0] or percolated[1] or percolated[2]) 
    return is_dead

def is_percolated(s, percolating_clusters) : 
    """
    Returns True if at least one cluster in percolating_clusters is truely percolated at least one axis direction among a, b, and c-axis.
    s : pymatgen.core.Structure
    percolating_clusters : list contains each clusters. (Ex : [[1,3,4,5,6], [7,8,2], [9]]) 
    """
    is_dead_structure = [is_dead_cluster(s, cluster) for cluster in percolating_clusters]
    if False in is_dead_structure : 
        return True
    else : 
        return False
    
def is_percolated_detail(s, percolating_clusters) : 
    """
    Returns percolation information in detail.
    Note that is_percolated_detail['percolated'] and is_percolated_detail['percolating_axis'] has same dimension.
    
    s : pymatgen.core.Structure
    percolating_clusters : list contains each clusters. (Ex : [[1,3,4,5,6], [7,8,2], [9]]) 
    """
    details = {}
        
    dead_clusters = [cluster for cluster in percolating_clusters if is_dead_cluster(s, cluster) == True]
    percolated_clusters = [cluster for cluster in percolating_clusters if cluster not in dead_clusters]

    percolated_axis = []
    for cluster in percolated_clusters : 
        axis = [axis for axis in ['a', 'b', 'c'] if is_cluster_percolated(s, cluster, axis) == True]
        percolated_axis.append(axis)
    
    if not percolated_clusters : 
        percolated_clusters.append(None)
    if not dead_clusters : 
        dead_clusters.append(None)
    if not percolated_axis : 
        percolated_axis.append(None)
    
    details['is_percolated'] = is_percolated(s, percolating_clusters)
    details['percolated'] = percolated_clusters
    details['dead'] = dead_clusters
    details['percolating_axis'] = percolated_axis
    
    return details

def get_num_percolated(clusters) : 
    """
    Returns total number of atoms contribute to percolation.
    clusters : list contains list of indexes of percolated atoms (Ex : [[1,2,6,8,10,13,16], [18, 19, 20, 22]])
    Note that 'clusters' is percolated, aware do not input 'Total cluster data' which contains dead clusters such as
    ([[51], [52, 59, 58, 55, 53], [54, 62, 56, 57, 63], [60], [61]]).
    For clarity, use is_percolated_detail(s, cluster)['percolated'].
    """
    num = 0
    for cluster in clusters : 
        if cluster is not None : 
            num += len(cluster)
    return num

# Random structure Mn percolation analysis

In [ ]:
# Generate random structures
num_structure = 1000
num_Mn = 102
num_Ti = 102
num_F = 0

basefile = '/home/debbang03/MnO_8x8x8.vasp' # mother structure
result_file = '/home/debbang03/random_structures.pkl'
structure_data = {}

for i in tqdm(range(num_structure)) : 
    s = Structure.from_file(basefile)
    s['Mn'] = 'Li'
    base_Li_index = [i for i in range(len(s)) if s[i].specie.name == 'Li'] 
    base_O_index = [i for i in range(len(s)) if s[i].specie.name == 'O'] 

    substitute_Mn_index = random.sample(base_Li_index, num_Mn) 
    remain_Li_index = [i for i in base_Li_index if i not in substitute_Mn_index]
    substitute_Ti_index = random.sample(remain_Li_index, num_Ti)
    
    substitute_F_index = random.sample(base_O_index, num_F) 

    for j in substitute_Mn_index : 
        s[j] = 'Mn'
    for j in substitute_Ti_index : 
        s[j] = 'Ti'
    for j in substitute_F_index : 
        s[j] = 'F'
    
    s = s.get_sorted_structure()
    structure_data['{}'.format(i+1)] = s
    
with open(result_file, 'wb') as f : 
    pickle.dump(structure_data, f)

In [ ]:
# Edge sharing analysis

with open('/home/debbang03/random_structures.pkl', 'rb') as f : 
    samples = pickle.load(f)

filename = '/home/debbang03/random_edge.json'

for i in tqdm(range(len(samples))) : 
    if os.path.isfile(filename) : 
        with open(filename, 'r') as f : 
            total_result = json.load(f)
    else : 
        total_result = {}
    
    s = samples['{}'.format(i+1)]
    percolation = get_percolating_cluster_radius_from_structure(s, percolating_species='Mn', radius=3.4) # edge+corner = 4.6, edge = 3.4 > 구조 보고 파악 
    result = is_percolated_detail(s, percolation)
    
    total_result['sample_{}'.format(i+1)] = result

    with open(filename, 'w') as f : 
        json.dump(total_result, f, indent=4)

In [ ]:
# Edge and corner sharing analysis

with open('/home/debbang03/random_structures.pkl', 'rb') as f : 
    samples = pickle.load(f)

filename = '/home/debbang03/random_edge_corner.json'

for i in tqdm(range(len(samples))) : 
    if os.path.isfile(filename) : 
        with open(filename, 'r') as f : 
            total_result = json.load(f)
    else : 
        total_result = {}
    
    s = samples['{}'.format(i+1)]
    percolation = get_percolating_cluster_radius_from_structure(s, percolating_species='Mn', radius=4.6) # edge+corner = 4.6, edge = 3.4 > 구조 보고 파악 
    result = is_percolated_detail(s, percolation)
    
    total_result['sample_{}'.format(i+1)] = result

    with open(filename, 'w') as f : 
        json.dump(total_result, f, indent=4)

# Canonical ensemble samples Mn percolation analysis

In [ ]:
# Edge sharing analysis

with open('/home/debbang03/test/3273K_10000thin.pkl', 'rb') as f : 
    samples = pickle.load(f)

filename = '/home/debbang03/3273K_edge.json'

for i in tqdm(range(len(samples))) : 
    if os.path.isfile(filename) : 
        with open(filename, 'r') as f : 
            total_result = json.load(f)
    else : 
        total_result = {}
    
    s = samples['{}'.format(i+1)]
    percolation = get_percolating_cluster_radius_from_structure(s, percolating_species='Mn', radius=3.4) # edge+corner = 4.6, edge = 3.4 > 구조 보고 파악 
    result = is_percolated_detail(s, percolation)
    
    total_result['sample_{}'.format(i+1)] = result

    with open(filename, 'w') as f : 
        json.dump(total_result, f, indent=4)

In [ ]:
# Edge and corner sharing analysis

with open('/home/debbang03/test/3273K_10000thin.pkl', 'rb') as f : 
    samples = pickle.load(f)

filename = '/home/debbang03/3273K_edge_corner.json'

for i in tqdm(range(len(samples))) : 
    if os.path.isfile(filename) : 
        with open(filename, 'r') as f : 
            total_result = json.load(f)
    else : 
        total_result = {}
    
    s = samples['{}'.format(i+1)]
    percolation = get_percolating_cluster_radius_from_structure(s, percolating_species='Mn', radius=4.6) # edge+corner = 4.6, edge = 3.4 > 구조 보고 파악 
    result = is_percolated_detail(s, percolation)
    
    total_result['sample_{}'.format(i+1)] = result

    with open(filename, 'w') as f : 
        json.dump(total_result, f, indent=4)

# Result json file analysis

In [ ]:
with open('/home/debbang03/test/3273K_10000thin.pkl', 'rb') as f : 
    sts = pickle.load(f)
sts['1'].composition

In [ ]:
filename = '/home/debbang03/3273K_edge_corner.json'

with open(filename, 'r') as f : 
    total_result = json.load(f)
    
total_tf = [total_result[sample]['is_percolated'] for sample in total_result.keys()]
num_percolated = [get_num_percolated(total_result[sample]['percolated']) for sample in total_result.keys()]

ratio_percolated = np.average([num_percolation/26 for num_percolation in num_percolated if num_percolation != 0])
percolated_Mn_per_O2 = np.average([2*num_percolation/512 for num_percolation in num_percolated if num_percolation != 0])

In [ ]:
len(total_result)

In [ ]:
percolated = [i for i in range(len(total_tf)) if total_tf[i] == True]
len(percolated)

In [ ]:
ratio_percolated

In [ ]:
percolated_Mn_per_O2

# Li percolation verification

In [ ]:
# Huiwen Ji et al, Nat. Commun. 10, 592 (2019)

def get_Li_percolation(s, tetra_st, r=2, tetra_species = 'H') : 
    """
    Return Li percolation data in Li/Mn/Ti/O DRX structure.
    Li percolation data contains Li4, Li3Mn1, Li3Ti1, Li2Mn2, Li2Mn1Ti1, and Li2Ti2 sites.
    Other configurations are treated as 'others'.
    s : DRX structure
    tetra_st : Rocksalt structure contains tetrahedral sites (default : H atom in tetra sites)
    """
    s = s.copy()
    
    sm = StructureMatcher(primitive_cell=False, comparator=FrameworkComparator(), ignored_species=[tetra_species])
    if sm.fit(s, tetra_st) == False : 
        raise Exception("""s and tetra_st are not matched. s and tetra_st should have same framework.
                          Current composition of s : {}
                          Current lattice parameters of s : {}
                          Current lattice angles of s : {}
                          ====================================
                          Current composition of tetra_st : {}
                          Current lattice parameters of tetra_st : {}
                          Current lattice angles of tetra_st : {}""".format(s.composition, s.lattice.abc, s.lattice.angles, tetra_st.composition, tetra_st.lattice.abc, tetra_st.lattice.angles))
    else : 
        pass

    tetra_sites = [site for site in tetra_st if site.specie.name == tetra_species]
    for site in tetra_sites : 
        s.append(species = site.specie, coords = site.frac_coords)
        
    li_percolation = []
    tetra_sites = [site for site in s if site.specie.name == tetra_species]
    for i, tetra_site in enumerate(tetra_sites) : 
        neighbors = s.get_neighbors(tetra_site, r=2)
        octa_neighbors = [site for site in neighbors if site.specie.name not in ['H', 'O']]
        li_percolation.append({"site_number" : "site_{}".format(i+1),
                               "site" : tetra_site, 
                               "neighbors" : octa_neighbors})
    
    metal_clusters = {"Li4":[], "Li3Mn1":[], "Li3Ti1":[], "Li2Mn2":[], "Li2Mn1Ti1":[], "Li2Ti2":[],
                      "Li1Mn3":[], "Li1Ti3":[], "Li1Mn2Ti1":[], 'Li1Mn1Ti2':[], 'Mn4':[], 'Ti4':[]}
    for i, data in enumerate(li_percolation) : 
        neighbor = data['neighbors']
        metals = [atom.specie.name for atom in neighbor]
        num_Li, num_Mn, num_Ti = metals.count('Li'), metals.count('Mn'), metals.count('Ti')

        if num_Li == 4 and num_Mn == 0 and num_Ti == 0 : 
            metal_clusters['Li4'].append(data)
        elif num_Li == 3 and num_Mn == 1 and num_Ti == 0 : 
            metal_clusters['Li3Mn1'].append(data)
        elif num_Li == 3 and num_Mn == 0 and num_Ti == 1 : 
            metal_clusters['Li3Ti1'].append(data)
        elif num_Li == 2 and num_Mn == 2 and num_Ti == 0 : 
            metal_clusters['Li2Mn2'].append(data)
        elif num_Li == 2 and num_Mn == 1 and num_Ti == 1 : 
            metal_clusters['Li2Mn1Ti1'].append(data)
        elif num_Li == 2 and num_Mn == 0 and num_Ti == 2 : 
            metal_clusters['Li2Ti2'].append(data)
        
        elif num_Li == 1 and num_Mn == 3 and num_Ti == 0: 
            metal_clusters['Li1Mn3'].append(data)
        elif num_Li == 1 and num_Mn == 0 and num_Ti == 3:
            metal_clusters['Li1Ti3'].append(data)
        elif num_Li == 1 and num_Mn == 2 and num_Ti == 1:
            metal_clusters['Li1Mn2Ti1'].append(data)
        elif num_Li == 1 and num_Mn == 1 and num_Ti == 2:
            metal_clusters['Li1Mn1Ti2'].append(data)
        elif num_Li == 1 and num_Mn == 4 and num_Ti == 0:
            metal_clusters['Mn4'].append(data)
        elif num_Li == 1 and num_Mn == 0 and num_Ti == 4:
            metal_clusters['Ti4'].append(data)
        
        
    return metal_clusters

In [ ]:
# t : MnO primitive structure with H atom at tetrahedral site
t = Structure.from_file('/home/debbang03/MnO_prim_tetra_H.vasp')
t

In [ ]:
t.make_supercell([8,8,8])
t.composition

In [ ]:
with open('/home/debbang03/test/3273K_10000thin.pkl', 'rb') as f : 
    samples = pickle.load(f)
    
ratio_Li4, ratio_Li3Mn1, ratio_Li3Ti1, ratio_Li2Mn2, ratio_Li2Mn1Ti1, ratio_Li2Ti2 = [], [], [], [], [], []    
ratio_Li1Mn3, ratio_Li1Ti3, ratio_Li1Mn2Ti1, ratio_Li1Mn1Ti2, ratio_Mn4, ratio_Ti4 = [], [], [], [], [], []

for i, s in enumerate(tqdm(samples)) : 
    result = get_Li_percolation(samples[s], t) # CE 구조, samples가 딕셔너리
    
    ratio_Li4.append(len(result['Li4'])/t.composition['H'])
    ratio_Li3Mn1.append(len(result['Li3Mn1'])/t.composition['H'])
    ratio_Li3Ti1.append(len(result['Li3Ti1'])/t.composition['H'])
    ratio_Li2Mn2.append(len(result['Li2Mn2'])/t.composition['H'])
    ratio_Li2Mn1Ti1.append(len(result['Li2Mn1Ti1'])/t.composition['H'])
    ratio_Li2Ti2.append(len(result['Li2Ti2'])/t.composition['H'])
    
    ratio_Li1Mn3.append(len(result['Li1Mn3'])/t.composition['H'])
    ratio_Li1Ti3.append(len(result['Li1Ti3'])/t.composition['H'])
    ratio_Li1Mn2Ti1.append(len(result['Li1Mn2Ti1'])/t.composition['H'])
    ratio_Li1Mn1Ti2.append(len(result['Li1Mn1Ti2'])/t.composition['H'])
    ratio_Mn4.append(len(result['Mn4'])/t.composition['H'])
    ratio_Ti4.append(len(result['Ti4'])/t.composition['H'])

In [ ]:
print("Li4 ratio : {}".format(np.mean(ratio_Li4)))
print()
print("Li3Mn1 ratio : {}".format(np.mean(ratio_Li3Mn1)))
print()
print("Li3Ti1 ratio : {}".format(np.mean(ratio_Li3Ti1)))
print()
print("Li2Mn2 ratio : {}".format(np.mean(ratio_Li2Mn2)))
print()
print("Li2Mn1Ti1 ratio : {}".format(np.mean(ratio_Li2Mn1Ti1)))
print()
print("Li2Ti2 ratio : {}".format(np.mean(ratio_Li2Ti2)))
print()
print("Li1Mn3 ratio : {}".format(np.mean(ratio_Li1Mn3)))
print()
print("Li1Ti3 ratio : {}".format(np.mean(ratio_Li1Ti3)))
print()
print("Li1Mn2Ti1 ratio : {}".format(np.mean(ratio_Li1Mn2Ti1)))
print()
print("Li1Mn1Ti2 ratio : {}".format(np.mean(ratio_Li1Mn1Ti2)))
print()
print("Mn4 ratio : {}".format(np.mean(ratio_Mn4)))
print()
print("Ti4 ratio : {}".format(np.mean(ratio_Ti4)))